In [6]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, count, when, lit, sum, row_number

StatementMeta(, 711f3870-325d-4b45-b33f-0743a773d20c, 8, Finished, Available, Finished)

In [2]:
silver_table_path = "abfss://Eolienne@onelake.dfs.fabric.microsoft.com/LH_Wind_Power_Silver.Lakehouse/Tables/dbo/wind_power"

df = spark.read.format("delta").load(silver_table_path)

print(f"📊 Données chargées : {df.count()} lignes")

StatementMeta(, 711f3870-325d-4b45-b33f-0743a773d20c, 4, Finished, Available, Finished)

📊 Données chargées : 3024 lignes


In [3]:
# Créer dim_date avec valeurs uniques
date_dim = (df
    .select("date", "day", "month", "quarter", "year")
    .distinct()
    .withColumnRenamed("date", "date_id")
)

print(f"📅 dim_date : {date_dim.count()} dates uniques")
date_dim.show(5)

StatementMeta(, 711f3870-325d-4b45-b33f-0743a773d20c, 5, Finished, Available, Finished)

📅 dim_date : 7 dates uniques
+----------+---+-----+-------+----+
|   date_id|day|month|quarter|year|
+----------+---+-----+-------+----+
|2024-06-17| 17|    6|      2|2024|
|2024-06-18| 18|    6|      2|2024|
|2024-06-15| 15|    6|      2|2024|
|2024-06-16| 16|    6|      2|2024|
|2024-06-19| 19|    6|      2|2024|
+----------+---+-----+-------+----+
only showing top 5 rows



In [4]:
# Créer dim_time avec valeurs uniques
time_dim = (df
    .select("time", "hour_of_day", "minute_of_hour", "second_of_minute", "time_period")
    .distinct()
    .withColumnRenamed("time", "time_id")
)

print(f"⏰ dim_time : {time_dim.count()} temps uniques")
time_dim.show(5)

StatementMeta(, 711f3870-325d-4b45-b33f-0743a773d20c, 6, Finished, Available, Finished)

⏰ dim_time : 144 temps uniques
+--------+-----------+--------------+----------------+-----------+
| time_id|hour_of_day|minute_of_hour|second_of_minute|time_period|
+--------+-----------+--------------+----------------+-----------+
|10:10:00|         10|            10|               0|    Morning|
|11:10:00|         11|            10|               0|    Morning|
|18:10:00|         18|            10|               0|    Evening|
|18:30:00|         18|            30|               0|    Evening|
|16:10:00|         16|            10|               0|  Afternoon|
+--------+-----------+--------------+----------------+-----------+
only showing top 5 rows



In [7]:
# Créer dim_turbine avec ID auto-incrémenté
turbine_dim = (df
    .select("turbine_name", "capacity", "location_name", "latitude", "longitude", "region")
    .distinct()
    .withColumn("turbine_id", 
        row_number().over(
            Window.orderBy("turbine_name", "capacity", "location_name")
        )
    )
)

print(f"🌬️ dim_turbine : {turbine_dim.count()} turbines")
turbine_dim.show()

StatementMeta(, 711f3870-325d-4b45-b33f-0743a773d20c, 9, Finished, Available, Finished)

🌬️ dim_turbine : 3 turbines
+------------+--------+-------------+--------+---------+--------+----------+
|turbine_name|capacity|location_name|latitude|longitude|  region|turbine_id|
+------------+--------+-------------+--------+---------+--------+----------+
|   Turbine A|    2200|   Location 1| 34.0522|-118.2437|Region A|         1|
|   Turbine B|    2000|   Location 2| 36.7783|-119.4179|Region B|         2|
|   Turbine C|    2500|   Location 3| 40.7128|  -74.006|Region C|         3|
+------------+--------+-------------+--------+---------+--------+----------+



In [8]:
# Créer dim_operational_status avec ID
operational_status_dim = (df
    .select("status", "responsible_department")
    .distinct()
    .withColumn("status_id", 
        row_number().over(
            Window.orderBy("status", "responsible_department")
        )
    )
)

print(f"📊 dim_operational_status : {operational_status_dim.count()} statuts")
operational_status_dim.show()

StatementMeta(, 711f3870-325d-4b45-b33f-0743a773d20c, 10, Finished, Available, Finished)

📊 dim_operational_status : 6 statuts
+--------------------+----------------------+---------+
|              status|responsible_department|status_id|
+--------------------+----------------------+---------+
|               Fault|           Engineering|        1|
|          Inspection|           Maintenance|        2|
|             Offline|            Operations|        3|
|              Online|            Operations|        4|
|Preventive Mainte...|           Maintenance|        5|
|Reactive Maintenance|           Maintenance|        6|
+--------------------+----------------------+---------+



In [9]:
# Joindre pour récupérer les clés étrangères
df_with_keys = (df
    .join(turbine_dim, 
          ["turbine_name", "capacity", "location_name", "latitude", "longitude", "region"], 
          "left")
    .join(operational_status_dim, 
          ["status", "responsible_department"], 
          "left")
)

StatementMeta(, 711f3870-325d-4b45-b33f-0743a773d20c, 11, Finished, Available, Finished)

In [10]:
# Table de faits avec uniquement clés et mesures
fact_table = (df_with_keys
    .select(
        "production_id",
        col("date").alias("date_id"),
        col("time").alias("time_id"),
        "turbine_id",
        "status_id",
        "wind_speed",
        "wind_direction",
        "energy_produced"
    )
)

print(f"📈 fact_wind_power : {fact_table.count()} mesures")
fact_table.show(10)


StatementMeta(, 711f3870-325d-4b45-b33f-0743a773d20c, 12, Finished, Available, Finished)

📈 fact_wind_power : 3024 mesures
+-------------+----------+--------+----------+---------+----------+--------------+---------------+
|production_id|   date_id| time_id|turbine_id|status_id|wind_speed|wind_direction|energy_produced|
+-------------+----------+--------+----------+---------+----------+--------------+---------------+
|         8857|2024-06-21|12:00:00|         1|        4|      4.86|             N|        1281.86|
|         8858|2024-06-21|12:00:00|         2|        4|     13.92|             W|         913.08|
|         8859|2024-06-21|12:00:00|         3|        4|      7.21|             N|        1614.72|
|         8860|2024-06-21|12:10:00|         1|        4|     16.08|            NE|         1052.2|
|         8861|2024-06-21|12:10:00|         2|        2|      7.81|             E|            0.0|
|         8862|2024-06-21|12:10:00|         3|        1|       3.6|             W|            0.0|
|         8863|2024-06-21|12:20:00|         1|        6|     17.34|         

In [12]:
base_path = "abfss://Eolienne@onelake.dfs.fabric.microsoft.com/LH_Wind_Power_Gold.Lakehouse/Tables/dbo"

gold_date_dim_path = f"{base_path}/dim_date"
gold_time_dim_path = f"{base_path}/dim_time"
gold_turbine_dim_path = f"{base_path}/dim_turbine"
gold_operational_status_dim_path = f"{base_path}/dim_operational_status"
gold_fact_table_path = f"{base_path}/fact_wind_power"

StatementMeta(, 711f3870-325d-4b45-b33f-0743a773d20c, 14, Finished, Available, Finished)

In [13]:
# Sauvegarder toutes les tables
date_dim.write.format("delta").mode("overwrite").save(gold_date_dim_path)
time_dim.write.format("delta").mode("overwrite").save(gold_time_dim_path)
turbine_dim.write.format("delta").mode("overwrite").save(gold_turbine_dim_path)
operational_status_dim.write.format("delta").mode("overwrite").save(gold_operational_status_dim_path)
fact_table.write.format("delta").mode("overwrite").save(gold_fact_table_path)

print("✅ Toutes les tables créées dans Gold !")

StatementMeta(, 711f3870-325d-4b45-b33f-0743a773d20c, 15, Finished, Available, Finished)

✅ Toutes les tables créées dans Gold !


In [14]:
from pyspark.sql.functions import count

print("=== 🔍 Vérification intégrité référentielle ===\n")

# Vérifier les dates
fact_dates = fact_table.select("date_id").distinct().count()
dim_dates = date_dim.count()
print(f"📅 Dates - Fait: {fact_dates}, Dim: {dim_dates}")

# Vérifier les turbines
fact_turbines = fact_table.select("turbine_id").distinct().count()
dim_turbines = turbine_dim.count()
print(f"🌬️ Turbines - Fait: {fact_turbines}, Dim: {dim_turbines}")

# Vérifier les clés nulles
null_check = fact_table.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in ["date_id", "time_id", "turbine_id", "status_id"]
])
print("\n❌ Clés nulles dans fact :")
null_check.show()

StatementMeta(, 711f3870-325d-4b45-b33f-0743a773d20c, 16, Finished, Available, Finished)

=== 🔍 Vérification intégrité référentielle ===

📅 Dates - Fait: 7, Dim: 7
🌬️ Turbines - Fait: 3, Dim: 3

❌ Clés nulles dans fact :
+-------+-------+----------+---------+
|date_id|time_id|turbine_id|status_id|
+-------+-------+----------+---------+
|      0|      0|         0|        0|
+-------+-------+----------+---------+



# Transformation Silver → Gold

## 📋 Objectif
Créer un modèle dimensionnel (star schema) optimisé pour l'analyse.

## 🌟 Modèle créé
- **4 dimensions** : Date, Time, Turbine, Operational Status
- **1 fait** : Production éolienne

## 📦 Dépendances
- **Input** : LH_Wind_Power_Silver.dbo.wind_power
- **Output** : LH_Wind_Power_Gold (5 tables)